In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\USER\\DataScience\\Projects\\End-To-End-Mushroom-with-MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\USER\\DataScience\\Projects\\End-To-End-Mushroom-with-MLFLOW'

## Entity setup

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

## src/mlProject/config/configuration.py

Configuration Manager Class


In [7]:
from src.Mushroom_classifier.constants import *
from src.Mushroom_classifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
        )

        return data_transformation_config
    

## Components for Data Transformation

In [10]:
import os 
from src.Mushroom_classifier import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd


In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting because this data is already cleaned up.

    def label_encoding_to_all_feature(self):
        data = pd.read_csv(self.config.data_path)

        # converting all categorical columns to numerical
        logger.info("converting all categorical columns to numerical using LabelEncoder")

        labelencoder = LabelEncoder()
        for column in data.columns:
            data[column] = labelencoder.fit_transform(data[column])
        logger.info("Columns datatype after converting to numerical")
        logger.info(f"{data.dtypes}")
        data.to_csv(os.path.join(self.config.root_dir, "data.csv"), index=False)
    

    def train_test_spliting(self):
        data = pd.read_csv(self.config.root_dir+"/data.csv")

        # Split the data into training and test sets. (0.80, 0.20) split.
        train, test = train_test_split(data, random_state = 42, test_size = .2)


        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)



## Data Transformation Pipeline

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.label_encoding_to_all_feature()
    data_transformation.train_test_spliting()

except Exception as e:
    raise e


[2025-09-24 00:44:44,396: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-24 00:44:44,403: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 00:44:44,409: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-24 00:44:44,412: INFO: common: created directory at: artifacts]
[2025-09-24 00:44:44,414: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-24 00:44:44,485: INFO: 168495426: converting all categorical columns to numerical using LabelEncoder]
[2025-09-24 00:44:44,527: INFO: 168495426: Columns datatype after converting to numerical]
[2025-09-24 00:44:44,531: INFO: 168495426: class                       int32
cap-shape                   int32
cap-surface                 int32
cap-color                   int32
bruises                     int32
odor                        int32
gill-attachment             int32
gill-spacing                int32
gill-size                   int32
gill-color              